In [ ]:
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np

Import Data

In [ ]:
county_cases_data = pd.read_csv("https://rajstorage11.blob.core.windows.net/test/us-counties.csv")
unemployment_data = pd.read_csv("https://rajstorage11.blob.core.windows.net/test/Unemployment.csv", index_col="FIPStxt")
county_area_data = pd.read_csv("https://rajstorage11.blob.core.windows.net/test/Counties.csv", index_col="GEOID")
population_data = pd.read_csv("https://rajstorage11.blob.core.windows.net/test/PopulationEstimates.csv", index_col="FIPStxt", encoding='latin-1')

In [ ]:
county_cases_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [ ]:
unemployment_data.head()

,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,...,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
FIPStxt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,"143,786,778","136,978,129","6,808,649",4.7,"144,839,532","136,455,911","8,383,621",5.8,"145,660,181","136,944,540","8,715,641",6.0,"146,725,005","138,614,038","8,110,967",5.5,"148,012,786","140,462,401","7,550,385",5.1,"150,223,730","143,262,292","6,961,438",4.6,"152,191,286","145,156,247","7,035,039",4.6,"153,761,298","144,860,522","8,900,776",...,"14,862,528",9.6,"154,606,324","140,765,822","13,840,502",9.0,"155,119,209","142,600,412","12,518,797",8.1,"155,485,689","144,018,150","11,467,539",7.4,"155,957,310","146,338,323","9,618,987",6.2,"156,798,981","148,515,185","8,283,796",5.0,"158,626,135","150,902,618","7,723,517",4.9,"160,158,564","153,183,461","6,975,103",4.4,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,"2,115,401","2,006,884","108,517",5.1,"2,106,161","1,981,919","124,242",5.9,"2,120,225","1,992,732","127,493",6.0,"2,136,458","2,014,889","121,569",5.7,"2,146,025","2,049,791","96,234",4.5,"2,167,809","2,080,233","87,576",4.0,"2,175,612","2,089,127","86,485",4.0,"2,176,489","2,053,477","123,012",...,"231,483",10.5,"2,202,670","1,990,413","212,257",9.6,"2,176,337","2,003,290","173,047",8.0,"2,174,000","2,017,043","156,957",7.2,"2,160,842","2,014,290","146,552",6.8,"2,161,481","2,030,005","131,476",6.0,"2,182,558","2,055,251","127,307",5.8,"2,186,869","2,090,373","96,496",4.4,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,"21,955","21,055",900,4.1,"22,094","21,035","1,059",4.8,"22,604","21,462","1,142",5.1,"23,218","22,103","1,115",4.8,"23,949","23,037",912,3.8,"24,398","23,585",813,3.3,"24,383","23,577",806,3.3,"24,687","23,420","1,267",...,"2,282",8.9,"25,836","23,677","2,159",8.4,"25,740","23,961","1,779",6.9,"25,810","24,205","1,605",6.2,"25,592","24,097","1,495",5.8,"25,652","24,321","1,331",5.0,"26,031","24,709","1,322",5.1,"26,075","25,062","1,013",3.9,"26,196","25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0
1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,"69,161","66,195","2,966",4.3,"69,169","65,691","3,478",5.0,"72,299","

Assign the starting day of the virus spread to every FIP.

In [ ]:
fips_case_start = county_cases_data.reset_index()
fips_case_start = fips_case_start[fips_case_start["cases"] > 0].groupby("fips").min().rename(columns={"date": "day0"})
fips_case_start["day0"] = fips_case_start["day0"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
fips_case_start.head()

,index,day0,county,state,cases,deaths
fips,,,,,,
1001.0,9480,2020-03-24,Autauga,Alabama,1,0
1003.0,1934,2020-03-14,Baldwin,Alabama,1,0
1005.0,28389,2020-04-03,Barbour,Alabama,1,0
1007.0,19658,2020-03-30,Bibb,Alabama,2,0
1009.0,10834,2020-03-25,Blount,Alabama,1,0


Establish FIPs Case History

In [ ]:
fips_case_history = county_cases_data[["date", "fips", "county", "state", "cases", "deaths"]]
fips_case_history.head()

,date,fips,county,state,cases,deaths
0,2020-01-21,53061.0,Snohomish,Washington,1,0
1,2020-01-22,53061.0,Snohomish,Washington,1,0
2,2020-01-23,53061.0,Snohomish,Washington,1,0
3,2020-01-24,17031.0,Cook,Illinois,1,0
4,2020-01-24,53061.0,Snohomish,Washington,1,0


In [ ]:
fips_case_history_extra = fips_case_history.copy()
fips_case_history_extra = fips_case_history_extra.join(fips_case_start[["day0"]], how="inner", on="fips")
fips_case_history_extra["date"] = fips_case_history_extra["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
fips_case_history_extra["days_spreading"] = fips_case_history_extra["date"] - fips_case_history_extra["day0"]
fips_case_history_extra["days_spreading"] = fips_case_history_extra["days_spreading"].apply(lambda x: x.days)
fips_case_history_extra.head()

,date,fips,county,state,cases,deaths,day0,days_spreading
0,2020-01-21,53061.0,Snohomish,Washington,1,0,2020-01-21,0
1,2020-01-22,53061.0,Snohomish,Washington,1,0,2020-01-21,1
2,2020-01-23,53061.0,Snohomish,Washington,1,0,2020-01-21,2
4,2020-01-24,53061.0,Snohomish,Washington,1,0,2020-01-21,3
7,2020-01-25,53061.0,Snohomish,Washington,1,0,2020-01-21,4


**Estimate R Values - SIR Model**

Add Each Fips to dictionary

In [ ]:
fips_ind = fips_case_history_extra[["date","fips","county","state","cases","deaths","days_spreading"]]
fips_ind = fips_ind.groupby("fips").filter(lambda x : len(x)>50)
Uniquefips = fips_ind.fips.unique()
DataFrameDict = {elem : pd.DataFrame for elem in Uniquefips}
for key in DataFrameDict.keys():
    DataFrameDict[key] = fips_ind[:][fips_ind.fips == key]

Run SIR Model - Estimate R values from gradient of infection curve

In [ ]:
from scipy.integrate import odeint
import numpy as np

for key in DataFrameDict.keys():

  #The total population of the county
  N = DataFrameDict[key].iloc[[-1]][["cases"]].astype(int).values[0][0]
  #The initial value of infected and removed population
  I0, R0 = 1, 0
  #Initial susceptible population
  S0 = N - I0 - R0
  beta = 0.142 # Use beta and gamma values from whole usa
  gamma = 0.056
  t = np.linspace(0, DataFrameDict[key].shape[0]+1, DataFrameDict[key].shape[0]+1)

  #ODE for SIR model
  def deriv(y, t, N, beta, gamma):
      S, I, R = y
      dSdt = -beta * S * I / N
      dIdt = beta * S * I / N - gamma * I
      dRdt = gamma * I
      return dSdt, dIdt, dRdt

  y0 = S0, I0, R0
  ret = odeint(deriv, y0, t, args=(N, beta, gamma))
  #Output for SIR model
  S, I, R = ret.T

  #Estimated R1 and R2 using slope 
  s2 = np.diff(I)/np.diff(t)
  s3 = s2*s2
  s2=(s2+2)/2
  DataFrameDict[key]['estimated_r_SIR1'] = s2
  DataFrameDict[key]['estimated_r_SIR2'] = s3

In [ ]:
DataFrameDict[1003].head()

,date,fips,county,state,cases,deaths,days_spreading,estimated_r_SIR1,estimated_r_SIR2
1934,2020-03-14,1003.0,Baldwin,Alabama,1,0,0,1.044685,0.007987
2309,2020-03-15,1003.0,Baldwin,Alabama,1,0,1,1.048685,0.009481
2738,2020-03-16,1003.0,Baldwin,Alabama,1,0,2,1.053036,0.011251
3214,2020-03-17,1003.0,Baldwin,Alabama,1,0,3,1.057768,0.013348
3760,2020-03-18,1003.0,Baldwin,Alabama,1,0,4,1.062911,0.015831


In [ ]:
county_case_history_r = pd.DataFrame()
for fips_code in DataFrameDict.keys():
  county_df = DataFrameDict[fips_code]
  county_case_history_r = county_case_history_r.append(county_df, ignore_index=True)

county_case_history_r.head()
county_case_history_r.nunique()

date                   142
fips                  2753
county                1648
state                   51
cases                 6619
deaths                1387
days_spreading         142
estimated_r_SIR1    174422
estimated_r_SIR2    174422
dtype: int64

**Estimate R Values - Timestep Ratio**

Function to estimate r from past cases

In [ ]:
def estimate_r_ratio(num_days):

  fips_r_base = fips_case_history_extra.copy()
  fips_r_base["date_delta"] = fips_r_base["date"].apply(lambda x: x+timedelta(days=num_days))
  fips_r_future = fips_r_base[["fips", "date", "cases", "county", "state"]].set_index(["fips", "date"], verify_integrity=True)

  fips_r_past = fips_r_base[["fips", "date", "cases","date_delta"]]
  fips_r_past = fips_r_past.rename(columns={"date": "from_date", "date_delta": "date", "cases": "past_cases"}).set_index(["fips", "date"])

  fips_r = fips_r_future.join(fips_r_past, on=["fips", "date"], how="inner")

  column_name = 'estimated_r_ratio{}'.format(num_days)
  fips_r["past_estimated_r"] = fips_r["cases"]/ fips_r["past_cases"]  # This estimates r for the past date

  fips_r_final = fips_r.copy().reset_index()
  fips_r_final = fips_r_final.drop('date',axis=1)
  fips_r_final  = fips_r_final.rename(columns={'from_date':'date','past_estimated_r':column_name})
  fips_r_final = fips_r_final.set_index(['fips','date'])
  return fips_r_final[column_name].to_frame()

In [ ]:
#Estimated R using the current and the past 14th day
estimated_r_ratio14 = estimate_r_ratio(14)
#Estimated R using the current and the past 8th day
estimated_r_ratio8 = estimate_r_ratio(8)
estimated_r_ratios = estimated_r_ratio14.join(estimated_r_ratio8)
estimated_r_ratios.tail(10)

estimated_r_ratio14  estimated_r_ratio8
fips    date                                               
48377.0 2020-05-26                  2.0                 1.0
        2020-05-27                  2.0                 1.0
48229.0 2020-05-26                  4.0                 2.0
        2020-05-27                 11.0                 3.0
19053.0 2020-05-27                  3.0                 1.0
29137.0 2020-05-27                  1.5                 1.0
29203.0 2020-05-27                  6.0                 5.0
41037.0 2020-05-27                  1.0                 1.0
48383.0 2020-05-27                  2.0                 2.0
56045.0 2020-05-27                  1.0                 1.0

In [ ]:
county_case_history_r.head()

,date,fips,county,state,cases,deaths,days_spreading,estimated_r_SIR1,estimated_r_SIR2
0,2020-01-21,53061.0,Snohomish,Washington,1,0,0,1.044893,0.008062
1,2020-01-22,53061.0,Snohomish,Washington,1,0,1,1.048949,0.009584
2,2020-01-23,53061.0,Snohomish,Washington,1,0,2,1.053370,0.011393
3,2020-01-24,53061.0,Snohomish,Washington,1,0,3,1.058190,0.013544
4,2020-01-25,53061.0,Snohomish,Washington,1,0,4,1.063444,0.016100


In [ ]:
county_case_history_final = county_case_history_r.join(estimated_r_ratios,on=['fips','date'],how='inner')
county_case_history_final.head()

,date,fips,county,state,cases,deaths,days_spreading,estimated_r_SIR1,estimated_r_SIR2,estimated_r_ratio14,estimated_r_ratio8
0,2020-01-21,53061.0,Snohomish,Washington,1,0,0,1.044893,0.008062,1.0,1.0
1,2020-01-22,53061.0,Snohomish,Washington,1,0,1,1.048949,0.009584,1.0,1.0
2,2020-01-23,53061.0,Snohomish,Washington,1,0,2,1.053370,0.011393,1.0,1.0
3,2020-01-24,53061.0,Snohomish,Washington,1,0,3,1.058190,0.013544,1.0,1.0
4,2020-01-25,53061.0,Snohomish,Washington,1,0,4,1.063444,0.016100,1.0,1.0


In [ ]:
# Export Estimated R Data
from google.colab import files

county_case_history_final_file = county_case_history_final.to_csv("county_case_history_final_file.csv")
files.download("county_case_history_final_file.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>